# Python code to classify text into known text categories

This code takes a .json file as input. The file should contain two columns - a "lyrics" column containing lyrics and a "song" column containing the song label.

Lexical similarity. 

Won't capture if similar words, not identical. 
Lexical similarity


In [55]:
import os
import csv
import json

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [67]:
#==============================================================================
# Load in text data, split into train/test 
#==============================================================================

# load in data
os.chdir("/Users/natashal/Projects/random_things/hack_cambridge/swift_lyrics/")
# lyrics = pd.read_json('swift.json') # works
lyrics = pd.read_json('dataset_swift3.json')

print lyrics

                                              lyrics          song
0  Cause, baby, now we got bad bloodYou know it u...     bad_blood
1  You know it used to be mad love (mad love)So t...     bad_blood
2  Nice to meet you, where you been?I could show ...   blank_space
3  Got a long list of ex-loversTheyll tell you Im...   blank_space
4  Cause were young and were recklessWell take th...   blank_space
5  I stay out too lateGot nothing in my brainThat...  shake_it_off
6  Im dancing on my own (dancing on my own)I make...  shake_it_off
7  Shes like Oh, my god! but Im just gonna shake....  shake_it_off


In [68]:
# what's the shape (rows, columns) of the data?
print(lyrics.shape); type(lyrics)
lyrics.head

(8, 2)


<bound method DataFrame.head of                                               lyrics          song
0  Cause, baby, now we got bad bloodYou know it u...     bad_blood
1  You know it used to be mad love (mad love)So t...     bad_blood
2  Nice to meet you, where you been?I could show ...   blank_space
3  Got a long list of ex-loversTheyll tell you Im...   blank_space
4  Cause were young and were recklessWell take th...   blank_space
5  I stay out too lateGot nothing in my brainThat...  shake_it_off
6  Im dancing on my own (dancing on my own)I make...  shake_it_off
7  Shes like Oh, my god! but Im just gonna shake....  shake_it_off>

In [69]:
# drop entries with null values, check shape afterwards
lyrics = lyrics.dropna(how='any')
lyrics.shape

# split my data into train and test sets
from sklearn.cross_validation import train_test_split
train, test = train_test_split(lyrics, test_size=0.2, random_state=42)
print train.shape; print test.shape

(6, 2)
(2, 2)


In [70]:
#==============================================================================
# Process description fields of train set
#==============================================================================

# tokenize the text using countvectoriser
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(lowercase=True, stop_words='english', strip_accents='unicode')
print count_vect.get_stop_words()

# if wanting to use n-grams
count_vect = CountVectorizer(analyzer='word', ngram_range=(1,2), lowercase=True, stop_words='english', strip_accents='unicode')

frozenset(['all', 'six', 'less', 'being', 'indeed', 'over', 'move', 'anyway', 'four', 'not', 'own', 'through', 'yourselves', 'fify', 'where', 'mill', 'only', 'find', 'before', 'one', 'whose', 'system', 'how', 'somewhere', 'with', 'thick', 'show', 'had', 'enough', 'should', 'to', 'must', 'whom', 'seeming', 'under', 'ours', 'has', 'might', 'thereafter', 'latterly', 'do', 'them', 'his', 'around', 'than', 'get', 'very', 'de', 'none', 'cannot', 'every', 'whether', 'they', 'front', 'during', 'thus', 'now', 'him', 'nor', 'name', 'several', 'hereafter', 'always', 'who', 'cry', 'whither', 'this', 'someone', 'either', 'each', 'become', 'thereupon', 'sometime', 'side', 'two', 'therein', 'twelve', 'because', 'often', 'ten', 'our', 'eg', 'some', 'back', 'up', 'go', 'namely', 'towards', 'are', 'further', 'beyond', 'ourselves', 'yet', 'out', 'even', 'will', 'what', 'still', 'for', 'bottom', 'mine', 'since', 'please', 'forty', 'per', 'its', 'everything', 'behind', 'un', 'above', 'between', 'it', 'neit

In [16]:
train

,lyrics,song
0,"'Cause, baby, now we got bad bloodYou know it ...",bad_blood
7,"She's like ""Oh, my god!"" but I'm just gonna sh...",shake_it_off
2,"Nice to meet you, where you been?I could show ...",blank_space
4,'Cause we're young and we're recklessWe'll tak...,blank_space
3,Got a long list of ex-loversThey'll tell you I...,blank_space
6,I'm dancing on my own (dancing on my own)I mak...,shake_it_off


In [37]:
# fit the count vectoriser
X_train_counts = count_vect.fit_transform(train.lyrics)
X_train_counts.shape
print count_vect.get_feature_names()[0:10]

# get term frequencies (tf), scale by inverse document frequenies (idf)
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer(use_idf=True, smooth_idf=True)

X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print X_train_tfidf.shape

# explore the matrix by converting back to dense format
dense=X_train_tfidf.todense()
dense[1:10, 1:10]

[u'alright', u'baby', u'baby couldn', u'baby got', u'baby just', u'baby queenfind', u'baby shake', u'babyand', u'babyand ll', u'bad']
(6, 368)


matrix([[ 0.04309764,  0.        ,  0.        ,  0.03112586,  0.        ,
          0.03112586,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.08774353],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.06364579,  0.06364579,  0.        ],
        [ 0.10244002,  0.        ,  0.        ,  0.        ,  0.1479679 ,
          0.        ,  0.12133579,  0.12133579,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ]])

(6, 368)


matrix([[ 0.04309764,  0.        ,  0.        ,  0.03112586,  0.        ,
          0.03112586,  0.        ,  0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.08774353],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.06364579,  0.06364579,  0.        ],
        [ 0.10244002,  0.        ,  0.        ,  0.        ,  0.1479679 ,
          0.        ,  0.12133579,  0.12133579,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ]])

In [30]:
#==============================================================================
# Training classifiers 
#==============================================================================
###################### 1) Start with naive bayes
from sklearn.naive_bayes import MultinomialNB
classifier_NB = MultinomialNB()
# train NB classifier
classifier_NB_fit = classifier_NB.fit(X_train_tfidf, train.song)
                    
# predict ratings on test set using model                    
test_counts = count_vect.transform(test.lyrics)
test_tfidf = tfidf_transformer.transform(test_counts)
predicted_nb = classifier_NB_fit.predict(test_tfidf)
print predicted_nb
print test.song

## get accuracy i.e. how often the predicted value eqausl the target values
print("NB accuracy", np.mean(predicted_nb == test.lyrics))

[u'blank_space' u'shake_it_off']
1       bad_blood
5    shake_it_off
Name: song, dtype: object
1    False
5    False
Name: lyrics, dtype: bool
('NB accuracy', 0.0)


In [33]:
####################### 2) Train the linear SVM 
from sklearn.linear_model import SGDClassifier
classifier_svm = SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42)
classifier_svm_fit = classifier_svm.fit(X_train_tfidf, train.song)                               
                               
# predict ratings on test set using model
predicted_svm = classifier_svm_fit.predict(test_tfidf) 

# get accuracy again, to compare to NB
print("SVM accuracy", np.mean(predicted_svm == test.song))

[u'bad_blood' u'shake_it_off']
1       bad_blood
5    shake_it_off
Name: song, dtype: object
('SVM accuracy', 1.0)


In [35]:
#==============================================================================
# Detailed performance metrics
#==============================================================================
# write out classification performance report
from sklearn import metrics
report = metrics.classification_report(test.song, predicted_svm)
print(report)

# write out confusion matrix
confusion = metrics.confusion_matrix(test.song, predicted_svm)
print(confusion)

             precision    recall  f1-score   support

  bad_blood       1.00      1.00      1.00         1
shake_it_off       1.00      1.00      1.00         1

avg / total       1.00      1.00      1.00         2

[[1 0]
 [0 1]]


In [34]:
### NOT TESTED
# DONT RUN THIS YET


def plot_confusion_matrix(confusion_matrix, title="Confusion matrix"):
    plt.matshow(confusion_matrix) 
    plt.xticks([0, 1, 2, 3, 4], [1, 2, 3, 4, 5])
    plt.yticks([0, 1, 2, 3, 4], [1, 2, 3, 4, 5])
    plt.colorbar()
    plt.title(title)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

plot_confusion_matrix(confusion)

# Normalize the confusion matrix by row (i.e by the number of samples
# in each class)
confusion_normalized = confusion.astype('float') / confusion.sum(axis=1)[:, np.newaxis]
print(confusion_normalized)
plot_confusion_matrix(confusion_normalized, title="Normalised confusion matrix")

NameError: name 'confusion' is not defined